In [31]:
import random
import copy
import time

### Class: `cell`
- **Purpose**: Represents a single cell in a Sudoku grid, encapsulating all properties and behaviors of a Sudoku cell.
- **Methods**:
  - `__init__(self, position)`: Initializes the cell with a set of all possible answers (1-9), an unset answer, a positional tuple for grid placement, and marks the cell as unsolved.
  - `remove(self, num)`: Removes a number from the possible answers list if the number is invalid based on Sudoku rules, automatically solving the cell if only one possibility remains.
  - `solvedMethod(self)`: Returns a boolean indicating whether the cell is solved.
  - `checkPosition(self)`: Returns the cell's positional tuple, helping identify its row, column, and block.
  - `returnPossible(self)`: Returns the list of possible answers for the cell.
  - `lenOfPossible(self)`: Returns the number of possible answers remaining for the cell.
  - `returnSolved(self)`: Returns the cell's answer if solved, otherwise returns 0.
  - `setAnswer(self, num)`: Directly sets the cell's answer if the given number is valid, marking it as solved.
  - `reset(self)`: Resets the cell to its initial state, clearing any set answer and marking it as unsolved.


In [32]:
class cell():
    # This class structure and methods are inspired by or based on a Sudoku solver implementation by Joe Karlsson.
    def __init__(self, position):
        self.possibleAnswers = [1, 2, 3, 4, 5, 6, 7, 8, 9]
        self.answer = None
        self.position = position
        self.solved = False

    def remove(self, num):
        if num in self.possibleAnswers and not self.solved:
            self.possibleAnswers.remove(num)
            if len(self.possibleAnswers) == 1:
                self.answer = self.possibleAnswers[0]
                self.solved = True

    def solvedMethod(self):
        return self.solved

    def checkPosition(self):
        return self.position

    def returnPossible(self):
        return self.possibleAnswers

    def lenOfPossible(self):
        return len(self.possibleAnswers)

    def returnSolved(self):
        return self.possibleAnswers[0] if self.solved else 0

    def setAnswer(self, num):
        if num in [1, 2, 3, 4, 5, 6, 7, 8, 9]:
            self.solved = True
            self.answer = num
            self.possibleAnswers = [num]
        else:
            raise ValueError("Invalid number")

    def reset(self):
        self.possibleAnswers = [1, 2, 3, 4, 5, 6, 7, 8, 9]
        self.answer = None
        self.solved = False


### Function: `emptySudoku()`
- **Purpose**: Creates a 9x9 grid of cell objects, each initialized in an unsolved state, representing an empty Sudoku board.

In [33]:
def emptySudoku():
    ans = []
    for x in range(1, 10):
        intz = 1 + (x-1)//3 * 3
        for y in range(1, 10):
            z = intz + (y-1)//3
            c = cell((x, y, z))
            ans.append(c)
    return ans

### Function: `printSudoku(sudoku)`
- **Purpose**: Prints the current state of the Sudoku grid in a formatted manner, enhancing readability with lines separating the 3x3 blocks.

In [34]:
def printSudoku(sudoku):
    # Print the top border of the grid
    print("+-------+-------+-------+")
    for row in range(9):
        if row % 3 == 0 and row != 0:
            print("|-------+-------+-------|")  # Print horizontal separators for blocks every three rows
        line = ""
        for col in range(9):
            num = sudoku[col + row * 9].returnSolved()
            num = ' ' if num == 0 else num  # Display unsolved cells as spaces
            if col % 3 == 0:
                line += "| "  # Start of a new block
            line += f"{num} "  # Add the number with a space
            if col == 8:
                line += "|"  # Close the Sudoku line at the end
        print(line)
    print("+-------+-------+-------+")  # Print the bottom border of the grid

### Function: `sudokuGen()`
- **Purpose**: Generates a complete, solvable Sudoku grid by filling in an empty grid while adhering to Sudoku rules.
- **Details**: Employs a heuristic of selecting cells with the fewest possible numbers to minimize complexity and potential errors during generation.

In [35]:
def sudokuGen():
     # Sudoku generation logic based on an algorithm by Joe Karlsson.
    # This implementation also uses a technique of selecting cells with the minimum possible answers to reduce complexity.
    
    cells = list(range(81))
    sudoku = emptySudoku()
    while cells:
        lowestNum = [sudoku[i].lenOfPossible() for i in cells]
        m = min(lowestNum)
        Lowest = [sudoku[i] for i in cells if sudoku[i].lenOfPossible() == m]
        choiceElement = random.choice(Lowest)
        choiceIndex = sudoku.index(choiceElement)
        cells.remove(choiceIndex)
        position1 = choiceElement.checkPosition()
        possibleValues = choiceElement.returnPossible()
        finalValue = random.choice(possibleValues)
        choiceElement.setAnswer(finalValue)
        for i in cells:
            position2 = sudoku[i].checkPosition()
            if position1[0] == position2[0] or position1[1] == position2[1] or position1[2] == position2[2]:
                sudoku[i].remove(finalValue)
    return sudoku

### Function: `sudokuChecker(sudoku)`
- **Purpose**: Validates a completed Sudoku grid to ensure all Sudoku rules are followed without any rule violations.
- **Details**: Checks every cell against every other cell for duplications in their respective rows, columns, and blocks.

In [36]:
def sudokuChecker(sudoku):
    # The approach to checking the validity of the Sudoku grid is based on techniques from Joe Karlsson.
    for i in range(len(sudoku)):  # Ensure 'i' is defined in a loop
        for n in range(len(sudoku)):
            if i != n:
                position1 = sudoku[i].checkPosition()
                position2 = sudoku[n].checkPosition()
                if position1[0] == position2[0] or position1[1] == position2[1] or position1[2] == position2[2]:
                    num1 = sudoku[i].returnSolved()
                    num2 = sudoku[n].returnSolved()
                    if num1 == num2:
                        return False
    return True

### Function: `perfectSudoku()`
- **Purpose**: Ensures the generation of a valid Sudoku puzzle by repeatedly generating and checking grids until a valid one is produced.

In [37]:
def perfectSudoku():
    result = False
    while not result:
        s = sudokuGen()
        result = sudokuChecker(s)
    return s

### Function: `solver(sudoku, f=0)`
- **Purpose**: Attempts to solve a Sudoku puzzle using a combination of backtracking and constraint propagation, suitable for puzzles with varying levels of difficulty.
- **Details**: Applies logical deductions to solve cells where possible, and uses guessing (backtracking) where necessary, tracking the number of guesses to assess difficulty.

In [38]:
def solver(sudoku, f=0):
     # This solver function is inspired by Joe Karlsson's method of backtracking.
    # Modifications and optimizations have been made to improve on code
    if f > 900:
        return False
    guesses = 0
    copy_s = copy.deepcopy(sudoku)
    cells = list(range(81))
    solvedCells = [i for i in cells if copy_s[i].lenOfPossible() == 1]
    while solvedCells:
        n = solvedCells.pop(0)
        cell = copy_s[n]
        position1 = cell.checkPosition()
        finalValue = cell.returnSolved()
        for i in cells:
            position2 = copy_s[i].checkPosition()
            if position1[0] == position2[0] or position1[1] == position2[1] or position1[2] == position2[2]:
                if copy_s[i].remove(finalValue) and copy_s[i].lenOfPossible() == 1 and i not in solvedCells:
                    solvedCells.append(i)
        cells.remove(n)
        if not cells:
            break
        if not solvedCells:
            lowestNum = [copy_s[i].lenOfPossible() for i in cells]
            m = min(lowestNum)
            lowest = [i for i in cells if copy_s[i].lenOfPossible() == m]
            randCell = random.choice(lowest)
            randGuess = random.choice(copy_s[randCell].returnPossible())
            copy_s[randCell].setAnswer(randGuess)
            solvedCells.append(randCell)
            guesses += 1
    if sudokuChecker(copy_s):
        level = 'Easy' if guesses == 0 else 'Medium' if guesses <= 2 else 'Hard' if guesses <= 7 else 'Insane'
        return copy_s, guesses, level
    else:
        return solver(sudoku, f + 1)


### Function: `solve(sudoku)`
- **Purpose**: Acts as a wrapper for the `solver` function, managing its outputs and providing a structured result including the solved grid, guess count, and a guess log.

In [39]:
def solve(sudoku):
    s = solver(sudoku)
    if s:
        return s
    else:
        return False

### Function: remove_numbers
- **Description**: Creates a Sudoku puzzle from a completed grid by strategically removing numbers.


In [40]:
def remove_numbers(sudoku, hints=36):
    """ Remove numbers to create a puzzle with 'hints' clues. """
    positions = list(range(81))
    random.shuffle(positions)
    removed = 81 - hints
    count = 0
    while count < removed:
        pos = positions.pop()
        row, col, _ = sudoku[pos].checkPosition()
        temp = sudoku[pos].returnSolved()
        sudoku[pos].reset()
        # Test if the puzzle remains solvable uniquely after this removal
        if not solve(copy.deepcopy(sudoku))[0]:
            sudoku[pos].setAnswer(temp)  # Put back the number if not solvable
        else:
            count += 1
    return sudoku


### Function: `prompt_to_solve(sudoku)`
- **Purpose**: Interfaces with the user, displaying the unsolved puzzle and asking for input to proceed with solving, thereafter showing the solved puzzle and related information.

In [41]:
def prompt_to_solve(sudoku):
    print("Here's the unsolved Sudoku puzzle:")
    printSudoku(sudoku)
    input("Type 'Finish' to solve: ")
    solved_puzzle, guesses, level = solve(copy.deepcopy(sudoku))
    print("Solved Puzzle:")
    printSudoku(solved_puzzle)
    print(f"Guesses: {guesses}")

### Function: `puzzleGen(difficulty)`
- **Purpose**: Generates a Sudoku puzzle of a specified difficulty by controlling the number of clues (hints) provided in the puzzle.

In [42]:
def puzzleGen(difficulty):
    """ Generate an unsolved puzzle based on difficulty. """
    solved_puzzle = perfectSudoku()
    if difficulty == 'Easy':
        return remove_numbers(solved_puzzle, hints=40)
    elif difficulty == 'Medium':
        return remove_numbers(solved_puzzle, hints=34)
    elif difficulty == 'Hard':
        return remove_numbers(solved_puzzle, hints=28)
    elif difficulty == 'Insane':
        return remove_numbers(solved_puzzle, hints=22)


### Function: `main(level)`
- **Purpose**: Main function to run the entire Sudoku puzzle generation and solving process based on the difficulty level chosen by the user.

In [43]:
def main(level):
    unsolved_puzzle = puzzleGen(level)
    prompt_to_solve(unsolved_puzzle)

### Function: `choose_level()`
- **Purpose**: Allows the user to select the difficulty level of the Sudoku puzzle, ensuring user input is valid before proceeding.

In [44]:
def choose_level():
    while True:
        level = input("Choose the level of difficulty (Easy/Medium/Hard/Insane): ").capitalize()
        if level in ['Easy', 'Medium', 'Hard', 'Insane']:
            break
        print("Invalid difficulty level. Please choose from Easy, Medium, Hard, or Insane.")
    return level

if __name__ == "__main__":
    level = choose_level()
    main(level)


Choose the level of difficulty (Easy/Medium/Hard/Insane):  Easy


Here's the unsolved Sudoku puzzle:
+-------+-------+-------+
| 8   5 | 2   7 |     1 |
|       | 4   6 | 5     |
| 7 6   | 9 1   |       |
|-------+-------+-------|
|       |       |   3 2 |
| 1 5 9 | 3 2 4 |   6   |
|       | 7   1 | 9 4 5 |
|-------+-------+-------|
|       | 1 7 9 | 3 5 6 |
|   3   |   4   | 7   9 |
| 9   7 | 6     |     8 |
+-------+-------+-------+


Type 'Finish' to solve:  Finish


Solved Puzzle:
+-------+-------+-------+
| 8 4 5 | 2 3 7 | 6 9 1 |
| 2 9 1 | 4 8 6 | 5 7 3 |
| 7 6 3 | 9 1 5 | 2 8 4 |
|-------+-------+-------|
| 6 7 4 | 5 9 8 | 1 3 2 |
| 1 5 9 | 3 2 4 | 8 6 7 |
| 3 8 2 | 7 6 1 | 9 4 5 |
|-------+-------+-------|
| 4 2 8 | 1 7 9 | 3 5 6 |
| 5 3 6 | 8 4 2 | 7 1 9 |
| 9 1 7 | 6 5 3 | 4 2 8 |
+-------+-------+-------+
Guesses: 41


## Credits and Acknowledgments

This implementation of a sudoku solver is inspired by Joe Karlsson's sudoku code. His code was used as a baseline to work with and improve off of.

- **Cell Class Design**: The structure and methods within the `cell` class, including but not limited to `remove`, `solvedMethod`, `setAnswer`, and `reset`, are inspired by Joe Karlsson's approach to managing Sudoku cells and their possible values.

- **Sudoku Grid Initialization**: The method used to initialize the Sudoku grid with appropriate box assignments based on row and column calculations is adapted from techniques popularized by Joe Karlsson

- **Solver Logic**: The solver's use of backtracking to resolve the Sudoku puzzles has been inspired by strategies used by Joe Karlsson.